In [51]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [52]:
BATCH_SIZE=100
n_classes = 10

In [53]:
# Define transforms 
transforms = transforms.Compose([transforms.Resize((32, 32)), 
                                transforms.ToTensor()])

# Download and create Datasets
train_dataset = torchvision.datasets.MNIST(
                                            root='./data/MNIST',
                                            train=True, 
                                            transform=transforms,
                                            download=True)
valid_dataset = torchvision.datasets.MNIST(
                                            root='./data/MNIST',
                                            train=False, 
                                            transform=transforms,
                                            download=True)

# Define the data Loaders 
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=BATCH_SIZE,
                         shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

In [182]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

In [184]:
model = LeNet5(n_classes=n_classes)

In [185]:
model

LeNet5(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)

### Using sequential model

In [186]:
# class LeNet5(nn.Module):
    
#     def __init__(self):
#         super(LeNet5, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0, bias=True)
#         self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)

#         self.activation = nn.Tanh()
#         self.pool = nn.AvgPool2d(kernel_size=2)
            
#         self.fc1 = nn.Linear(400, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.out = nn.Linear(84, n_classes)
    
#     def forward(self, x):
#         x = self.conv1(x) 
#         x = self.activation(x)
#         x = self.pool(x)
        
#         x = self.conv2(x)
#         x = self.activation(x)
#         x = self.pool(x)
        
#         x = torch.flatten(x, start_dim=1)        
        
#         x = self.fc1(x)
#         x = self.activation(x)
#         x = self.fc2(x)
#         x = self.activation(x)
#         logits = self.out(x)

#         probs = F.softmax(logits, dim=0)
#         return logits, probs
        
        

In [187]:
# batch = next(iter(train_loader))

In [188]:
# images, labels = batch
# images.shape => torch.Size([100, 1, 32, 32])

In [215]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [220]:
def accuracy_function(preds, labels):
    total_samples = len(labels)
    no_of_accurate_preds = preds.argmax(1).eq(labels).sum().item()
    accuracy = no_of_accurate_preds/total_samples
    return accuracy

In [221]:
EPOCHS = 2
for e in range(EPOCHS):
    for images, labels in train_loader:
        logits, preds = model(images)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    accuracy = accuracy_function(preds, labels)
    print('Epoch:{} \t Loss:{:.3F} \t Accuracy:{:.2F}'.format(e +1, loss.item(), accuracy))

Epoch:1 	 Loss:5.778 	 Accuracy:0.49
Epoch:2 	 Loss:6.089 	 Accuracy:0.47


In [192]:
valid_sample = next(iter(valid_loader))

In [193]:
images, labels = valid_sample

In [194]:
images.shape

torch.Size([100, 1, 32, 32])

In [195]:
_, preds = model(images)

In [196]:
preds.shape

torch.Size([100, 10])

In [197]:
labels

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9])

In [198]:
preds.argmax(1)

tensor([7, 2, 1, 0, 4, 1, 4, 4, 0, 9, 0, 0, 9, 0, 1, 5, 9, 7, 8, 4, 9, 6, 2, 5,
        9, 0, 7, 9, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 3, 1, 1, 7, 4, 2, 3, 3, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 8, 1, 7, 3, 2, 4, 7, 9, 6, 2, 7, 8, 4, 7, 5, 6, 1, 3, 6, 8, 3, 1, 9,
        3, 4, 6, 9], grad_fn=<NotImplemented>)

In [205]:
accuracy_function(preds, labels)

82.0